In [17]:
fivearray = np.array(5, float)
fivearray
np.linalg.norm(fivearray)

5.0

In [18]:
np.linalg.norm(np.array([1,2,3]))

3.7416573867739413

In [5]:
import cv2
import numpy as np
import mediapipe as mp
import time
from collections import deque

# ----------------- CONFIG -----------------
CAM_INDEX = 0 # choose camera
DISPLAY_MIRROR = True  # <- show mirrored preview, but keep logic non-mirrored

# Angle thresholds (degrees)
ANGLE_UP = 75
ANGLE_DOWN = 150

# Smoothing
ANGLE_EMA_ALPHA = 0.2
USE_MEDIAN_WINDOW = 5

# Debounce
HOLD_FRAMES = 2

# https://medium.com/@codetrade/implementation-of-human-pose-estimation-using-mediapipe-23a57968356b
# To initialize the pose estimation object we will call the pose class from mediapipe.solutions.pose and create an object 
# named mp_pose. You can replace the class name as per your requirements.
mp_pose = mp.solutions.pose


####################################################################################################################################
def angle_between(a, b, c):
    # a for shoulder, b for elbow, c for wrist
    # a, b, c are (x, y) coordinates (coordinate pairs) respectively, convert them to a vector of two elements respectively,
    # convert them to vectors so we can perform vector calculus
    a = np.array(a, float); b = np.array(b, float); c = np.array(c, float)
    print("shoulder = " + str(a) + ", elbow = " + str(b) + ", wrist = " + str(c))
    
    # calculate difference in coordinate
    ba = a - b; bc = c - b
    print("shoulder - elbow = " + str(ba) + ", elbow - wrist = " + str(bc))

    # np.linalg.norm(np.array[x, y]) = sqrt(x^2 + y^2)
    # find hypotenuse using pythagorean theorem
    # + 1e-8 to avoid division by 0 in cosang
    n1 = np.linalg.norm(ba) + 1e-8; n2 = np.linalg.norm(bc) + 1e-8
    print("hypotenuse (shoulder - elbow) = " + str(n1) + ", hypotenuse (elbow - wrist) = " + str(n2))
    
    # np.clip limits the values in an array
    # e.g. if interval [0, 1] is specified, then numbers smaller than 0 becomes 0, numbers large than 1 becomes 1.
    # right here, we calculate (np.dot(ba, bc))/(n1 * n2), subjected to clip interval [-1.0, 1.0]
    '''
    Using dot-product identity, 
    ba ⋅ bc = |ba||bc|cos(θ) => cos(θ) = (ba ⋅ bc)/(|ba||bc|)
    to get angle between two vectors.
    
    |u| = sqrt(u_x^2 + u_y^2)
    u.v = (u_x * v_x) + (u_y + v_y) = [u_x, u_y] ⋅ [v_x, v_y]^T
    '''
    cosang = np.clip(np.dot(ba, bc) / (n1 * n2), -1.0, 1.0)
    print("cosang = " + str(cosang))

    # cos(theta) = adjacent/hypotenuse
    # theta = arccos(adjacent/hypotenuse)
    print("theta = " + str(np.arccos(cosang)))
    return np.degrees(np.arccos(cosang))

####################################################################################################################################3
def get_arm_landmarks_xy(landmarks, w, h, which):

    # https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker
    # The MediaPipe Pose Landmarker task lets you detect landmarks of human bodies in an image or video. 
    L = mp_pose.PoseLandmark # 'L' for Landmark
   
    if which == "right":
        # Basically, something like "L.RIGHT_SHOULDER.value" is just a predefined index of a body landmark in PoseLandmark.
        s, e, wri = L.RIGHT_SHOULDER.value, L.RIGHT_ELBOW.value, L.RIGHT_WRIST.value
        print("L.RIGHT_SHOULDER.value = " + str(L.RIGHT_SHOULDER.value) + ", L.RIGHT_ELBOW.value = " +  str(L.RIGHT_ELBOW.value)
             + ", L.RIGHT_WRIST.value = " + str(L.RIGHT_WRIST.value))
    else:
        s, e, wri = L.LEFT_SHOULDER.value, L.LEFT_ELBOW.value, L.LEFT_WRIST.value
        print("L.LEFT_SHOULDER.value = " + str(L.LEFT_SHOULDER.value) + ", L.LEFT_ELBOW.value = " +  str(L.LEFT_ELBOW.value)
             + ", L.LEFT_WRIST.value = " + str(L.LEFT_WRIST.value))

    # get the 3D coordinates and light visibility index for the landmarks of shoulder, elbow, wrist
    S, E, W = landmarks[s], landmarks[e], landmarks[wri]
    print("3D Coordinates and Visibility of Landmarks: " + "S = " + str(S) + ", E = " + str(E) + ", W = " + str(W))

    # function_name = lambda <argument>: (<operations>)
    to_xy = lambda lm: (lm.x * w, lm.y * h)
    # Basically, extracts the (x, y) coordinates, multiply x by w and multiply y by h to convert
    '''
    MediaPipe gives landmark coordinates normalized to the image:
    lm.x and lm.y are floats ~in [0, 1] measured as a fraction of the frame width/height (origin at the top-left, +x to the right, +y downward).

    Quick example:
    If your frame is 1280×720 and a wrist has (lm.x=0.25, lm.y=0.50), then
    x_px = 0.25 * 1280 = 320
    y_px = 0.50 * 720 = 360
    → pixel ≈ (320, 360).
    '''
    
    return to_xy(S), to_xy(E), to_xy(W)
    
####################################################################################################################################
def mirror_xy(pt, width):
    """Mirror an (x,y) pixel coordinate for drawing on the mirrored view."""
    x, y = pt
    return (int(width - 1 - x), int(y))
'''
- Images use pixel indices x = 0, ..., width-1 from left → right.
- A horizontal mirror maps each x to its symmetric partner:
  x′ = (width − 1) − x
    So:
    ⋅ Left edge x=0 → x' = width-1 (right edge)
    ⋅ Right edge x=width-1 → x' = 0 (left edge)
    ⋅ Centerline x=(width-1)/2 stays the same
- y is unchanged because we’re only flipping left↔right.
- int(...) ensures the result is an integer pixel coordinate for OpenCV drawing.
'''

####################################################################################################################################
class ArmTracker:

    # constructor for ArmTracker class object
    '''
    __init__ method in Python is a constructor. 
    Basically you call a constructor method to create its class object.
    ____________________________________________
    Example: 
    
    class Person:
        def __init__(self, name, age):
            self.name = name
            self.age = age

    p1 = Person("Alice", 30)
    p2 = Person("Bob", 25)
    
    print(p1.name, p1.age)  # Alice 30
    print(p2.name, p2.age)  # Bob 25
    ____________________________________________
    
    '''
    def __init__(self, name): # Basically name is either the label "Left" or "Right"
        self.name = name
        self.state = "down"
        self.reps = 0 # count number of reps
        self.ema = None
        self.hist = deque(maxlen=USE_MEDIAN_WINDOW if USE_MEDIAN_WINDOW > 0 else 1)
        self.up_hold = 0
        self.down_hold = 0
        self.angle_vis_deg = None
        self.elbow_xy = None

    def update(self, raw_angle_deg, elbow_xy):
        # Add the newest elbow angle (in degrees) into a short rolling list (self.hist).
        self.hist.append(raw_angle_deg)

        # If median filtering is enabled, replace the raw angle with the median of the last N 
        # angles (kills 1-frame glitches). Otherwise, just use the raw value.
        angle_deg = float(np.median(self.hist)) if USE_MEDIAN_WINDOW > 0 else raw_angle_deg


        '''
        Smooth the (possibly median-filtered) angle with an Exponential Moving Average (EMA):
        first frame: set EMA to the current angle,
        later frames: blend current angle with previous EMA using ANGLE_EMA_ALPHA (α).
        bigger α → reacts faster, less smooth; smaller α → smoother, slower.
        Store the smoothed angle in angle_vis_deg (used for logic and for drawing on screen).
        Remember the elbow pixel location (so you can put the angle text near it).
        '''
        self.ema = angle_deg if self.ema is None else ANGLE_EMA_ALPHA * angle_deg + 
                   (1-ANGLE_EMA_ALPHA) * self.ema
        self.angle_vis_deg = self.ema
        self.elbow_xy = elbow_xy


        
        '''
        If the smoothed angle is large enough (≥ ANGLE_DOWN, i.e., arm looks extended), 
        increase down_hold.
        Otherwise, reset down_hold.
        Those “hold” counters are a debounce: require the condition to be true for several 
        frames (HOLD_FRAMES) so tiny jitters don’t flip state.
        '''
        if self.angle_vis_deg <= ANGLE_UP: self.up_hold += 1
        else: self.up_hold = 0

        if self.angle_vis_deg >= ANGLE_DOWN: self.down_hold += 1
        else: self.down_hold = 0

        if self.state == "down" and self.up_hold >= HOLD_FRAMES:
            self.state = "up"
        elif self.state == "up" and self.down_hold >= HOLD_FRAMES:
            self.state = "down"; self.reps += 1

####################################################################################################################################
def main():
    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        print("Could not open webcam."); return

    pose = mp_pose.Pose(
        static_image_mode=False, model_complexity=1,
        smooth_landmarks=True, enable_segmentation=False,
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    )

    left = ArmTracker("Left")
    right = ArmTracker("Right")
    t_prev = time.time()

    print("Press 'q' to quit.")
    while True:
        ok, frame_bgr = cap.read()
        if not ok: break

        h, w = frame_bgr.shape[:2]

        # process on the real non-mirrored frame
        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        res = pose.process(rgb)

        # Prepare a VIEW frame (mirrored only for display)
        view = cv2.flip(frame_bgr, 1) if DISPLAY_MIRROR else frame_bgr.copy()

        if res.pose_landmarks:
            # RIGHT arm
            try:
                rS, rE, rW = get_arm_landmarks_xy(res.pose_landmarks.landmark, w, h, "right")
                r_angle = angle_between(rS, rE, rW)
                right.update(r_angle, (int(rE[0]), int(rE[1])))

                # draw short segment on VIEW (mirror coords if needed)
                rpts = [tuple(map(int, rS)), tuple(map(int, rE)), tuple(map(int, rW))]
                if DISPLAY_MIRROR: rpts = [mirror_xy(p, w) for p in rpts]
                cv2.circle(view, rpts[0], 6, (0, 255, 255), -1)
                cv2.circle(view, rpts[1], 8, (0, 165, 255), -1)
                cv2.circle(view, rpts[2], 6, (0, 255, 255), -1)
                cv2.line(view, rpts[0], rpts[1], (0, 200, 0), 3)
                cv2.line(view, rpts[1], rpts[2], (0, 200, 0), 3)
            except Exception:
                pass

            # LEFT arm
            try:
                lS, lE, lW = get_arm_landmarks_xy(res.pose_landmarks.landmark, w, h, "left")
                l_angle = angle_between(lS, lE, lW)
                left.update(l_angle, (int(lE[0]), int(lE[1])))

                lpts = [tuple(map(int, lS)), tuple(map(int, lE)), tuple(map(int, lW))]
                if DISPLAY_MIRROR: lpts = [mirror_xy(p, w) for p in lpts]
                cv2.circle(view, lpts[0], 6, (0, 255, 255), -1)
                cv2.circle(view, lpts[1], 8, (0, 165, 255), -1)
                cv2.circle(view, lpts[2], 6, (0, 255, 255), -1)
                cv2.line(view, lpts[0], lpts[1], (0, 200, 0), 3)
                cv2.line(view, lpts[1], lpts[2], (0, 200, 0), 3)
            except Exception:
                pass

        # FPS
        t = time.time()
        fps = 1.0 / max(t - t_prev, 1e-6); t_prev = t

        # Angle labels near elbows on VIEW
        if right.angle_vis_deg is not None and right.elbow_xy is not None:
            rxy = mirror_xy(right.elbow_xy, w) if DISPLAY_MIRROR else right.elbow_xy
            cv2.putText(view, f"R:{int(right.angle_vis_deg)}°", (rxy[0]+10, rxy[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        if left.angle_vis_deg is not None and left.elbow_xy is not None:
            lxy = mirror_xy(left.elbow_xy, w) if DISPLAY_MIRROR else left.elbow_xy
            cv2.putText(view, f"L:{int(left.angle_vis_deg)}°", (lxy[0]+10, lxy[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

        # HUD (top-left is fine either way)
        cv2.putText(view,
                    f"Left Reps: {left.reps} (state:{left.state})   Right Reps: {right.reps} (state:{right.state})",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 2)
        cv2.putText(view, f"Up<{ANGLE_UP}°, Down>{ANGLE_DOWN}°   FPS:{fps:.1f}",
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200,200,200), 2)

        cv2.imshow("Dual-Arm Curl Detector (Mirrored View Only)", view)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    pose.close()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Press 'q' to quit.
L.RIGHT_SHOULDER.value = 12, L.RIGHT_ELBOW.value = 14, L.RIGHT_WRIST.value = 16
3D Coordinates and Visibility of Landmarks: S = x: 0.160929024
y: 1.01446092
z: -1.57490265
visibility: 0.944450855
, E = x: 0.0610180497
y: 1.30068588
z: -1.68949533
visibility: 0.398808122
, W = x: 0.241609842
y: 1.06212926
z: -1.79551065
visibility: 0.156863496

S.x * w = 0.16092902421951294 * 640 = 102.99457550048828

S.y * h = 1.0144609212875366 * 480 = 486.9412422180176

shoulder = [102.9945755  486.94124222], elbow = [ 39.05155182 624.32922363], wrist = [154.63029861 509.82204437]
shoulder - elbow = [  63.94302368 -137.38798141], elbow - wrist = [ 115.5787468  -114.50717926]
hypotenuse (shoulder - elbow) = 151.539327297615, hypotenuse (elbow - wrist) = 162.69708299858308
cosang = 0.9378364896924541
theta = 0.35445297340958287
L.LEFT_SHOULDER.value = 11, L.LEFT_ELBOW.value = 13, L.LEFT_WRIST.value = 15
3D Coordinates and Visibility of Landmarks: S = x: 0.710974216
y: 1.03657389
z: -